#Day 99

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from iso3166 import countries
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.2f}'.format
df_data = pd.read_csv('mission_launches.csv')

## Preliminary Data Exploration

* What is the shape of `df_data`?
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
print(f'shape of the dataframe: {df_data.shape}')
print(df_data.columns)
print(f'Duplicados: {df_data.duplicated().sum()}')
print(f'Null values: {df_data.isna().sum()}')
print(df_data.sample())

## Data Cleaning - Check for Missing Values and Duplicates

---



Removing columns containing junk data.

In [ ]:
useful_data = df_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0']) #remove unused columns
print(useful_data.shape)
contain_price_df = useful_data.dropna(subset=['Price']) #create a price dataframe
print(contain_price_df.shape)

In [ ]:
useful_data.describe() #just to know the normal d

## Number of Launches per Company

Chart that shows the number of space mission launches by organisation.

In [ ]:
plt.figure(figsize=(12, 6))
useful_data['Organisation'].value_counts().plot(kind='bar')

plt.title('Number of Space Mission Launches by Organisation')
plt.xlabel('Organisation')
plt.ylabel('Number of Launches')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()


## Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned?

In [ ]:
status_count = useful_data['Rocket_Status'].value_counts()
#print(status_count)

plt.figure(figsize=(6, 6))
plt.pie(status_count, labels=status_count.index, autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff'])
plt.title('Rocket Status: Active vs Retired')
plt.axis('equal')

plt.show()

## Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [ ]:
missions_status = useful_data['Mission_Status'].value_counts()

success_count = missions_status.get('Success', 0)
fail_count = missions_status.sum() - success_count #all that were not sucessful

labels = ['Successful', 'Failed']
counts = [success_count, fail_count]

plt.figure(figsize=(6, 4))
plt.bar(labels, counts, color=['green', 'red'])

plt.title('Mission Outcomes: Success vs Failure')
plt.ylabel('Number of Missions')
plt.tight_layout()
plt.show()

## How Expensive are the Launches?

histogram to visualise the distribution. The price column is given in USD millions.

In [ ]:
contain_price_df['Price'] = pd.to_numeric(contain_price_df['Price'], errors='coerce')#transform str price into numbered price
filtered_prices = [price for price in contain_price_df['Price'] if price <= 300] #select only under 300 million
bins = np.arange(0, 325, 25)  # 0, 25, 50, ..., 300

plt.figure(figsize=(10, 6))
plt.hist(filtered_prices, bins=bins, color='skyblue', edgecolor='black')

plt.title('Distribution of Launch Prices (≤ 300 Million USD)', fontsize=14)
plt.xlabel('Launch Price (Million USD)', fontsize=12)
plt.ylabel('Number of Launches', fontsize=12)
plt.xticks(bins)
plt.grid(axis='y', linestyle='--', alpha=0.6)

for i in range(len(bins)-1):
    count = ((filtered_prices >= bins[i]) & (filtered_prices < bins[i+1])).sum()
    plt.text((bins[i]+bins[i+1])/2, count + 1, str(count), ha='center', fontsize=9)

plt.tight_layout()
plt.show()


##Choropleth Map to Show the Number of Launches by Country

Need to use a 3 letter country code for each country.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


In [ ]:
import re

#extrair nome do país da string de localização
def extract_country_from_location(location):
    # regex para pegar tudo depois da ultima virgula
    match = re.search(r',\s*([^,]+)$', location)
    if match:
        return match.group(1).strip()  #extrai e croppa o nome
    return location  #se nao encontrar a virgula retorna tudo

#corrigir nome de paises
def correct_country_name(country):
    country_mapping = {
        'Russia': 'Russian Federation',
        'New Mexico': 'USA',
        'Yellow Sea': 'China',
        'Shahrud Missile Test Site': 'Iran',
        'Pacific Missile Range Facility': 'USA',
        'Barents Sea': 'Russian Federation',
        'Gran Canaria': 'USA',
        'Iran': 'Iran',
        'North Korea': 'North Korea',
        'Pacific Ocean': 'USA',
        'South Korea': 'South Korea'
    }
    return country_mapping.get(country, country)

#converte nome do pais em ISO
def get_country_alpha3(country_name):
    try:
        return countries.get(country_name).alpha3
    except KeyError:
        #print(f"Não foi possível encontrar o código para o país: {country_name}")
        return None  # se nao recuperar o ISO nao recebe

#extração e correcao de paises
useful_data['Extracted_Country'] = useful_data['Location'].apply(extract_country_from_location)
useful_data['Corrected_Country'] = useful_data['Extracted_Country'].apply(correct_country_name)
useful_data['Country_Code'] = useful_data['Corrected_Country'].apply(get_country_alpha3)


#nova coluna com os isos
useful_data['Country_Code'] = useful_data['Corrected_Country'].apply(get_country_alpha3)

#ver quais paises nao foram transformados
missing_countries = useful_data[useful_data['Country_Code'].isnull()]
#print(f"Países com código não encontrado: \n{missing_countries['Corrected_Country'].unique()}")

#lançamentos por ISO
launches_by_country = useful_data['Country_Code'].value_counts().reset_index()
launches_by_country.columns = ['Country_Code', 'Launch_Count']

#printar
print(launches_by_country.head())


In [ ]:
useful_data.head()

In [ ]:
fig = px.choropleth(launches_by_country,
                    locations='Country_Code',
                    color='Launch_Count',
                    hover_name='Country_Code',
                    color_continuous_scale='matter',
                    labels={'Launch_Count': 'Number of Launches'},
                    title='Number of Launches by Country')

fig.update_geos(showcoastlines=True, coastlinecolor="Black", showland=True, landcolor="white")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})  # Ajuste de margens
fig.show()

##Choropleth Map to Show the Number of Failures by Country

In [ ]:
failures_only = useful_data[useful_data['Mission_Status'] != 'Success'] #everythint that isnt sucess

failures_by_country = failures_only['Country_Code'].value_counts().reset_index()
failures_by_country.columns = ['iso_alpha', 'failures']

failures_by_country = failures_by_country.dropna(subset=['iso_alpha'])

fig = px.choropleth(
    failures_by_country,
    locations='iso_alpha',
    color='failures',
    color_continuous_scale='Reds',
    title='Falhas de Missão por País'
)

fig.update_layout(geo=dict(showframe=False, projection_type='equirectangular'))
fig.show()

##Plotly Sunburst Chart of the countries, organisations, and mission status.

In [ ]:
fig = px.sunburst(
    useful_data,
    path=['Corrected_Country', 'Organisation', 'Mission_Status'],
    values=None,  #all ocurrences
    color='Mission_Status',
    title='Missões por País, Organização e Status'
)

fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

## Total Amount of Money Spent by Organisation on Space Missions

In [ ]:
contain_price_df['Price'] = pd.to_numeric(contain_price_df['Price'], errors='coerce')

total_spent_by_org = contain_price_df.groupby('Organisation')['Price'].sum().reset_index()

total_spent_by_org = total_spent_by_org.sort_values(by='Price', ascending=False)


fig = px.bar(
    total_spent_by_org,
    x='Organisation',
    y='Price',
    title='Total Gasto por Organização em Missões Espaciais',
    labels={'Price': 'Gasto Total (em milhões de dólares)'},
    text_auto='.2s'
)

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_tickformat = "$,.2s",
    showlegend=False
)

fig.show()

## Amount of Money Spent by Organisation per Launch

In [ ]:
contain_price_df['Price'] = pd.to_numeric(contain_price_df['Price'], errors='coerce')
total_spent_by_org_launch = contain_price_df.groupby(['Organisation', 'Date'])['Price'].sum().reset_index()

fig = px.bar(
    total_spent_by_org_launch,
    x='Organisation',
    y='Price',
    color='Organisation',
    title='Distribuição de Gasto por Lançamento e Organização',
    labels={'Price': 'Gasto Total (em milhões de dólares)', 'Organisation': 'Organização'},
    text='Price'
)

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_tickformat="$,.2s",
    showlegend=False
)
fig.show()

## Chart of the Number of Launches per Year

In [ ]:
useful_data['Date'] = pd.to_datetime(useful_data['Date'], errors='coerce')
useful_data['Year'] = useful_data['Date'].dt.year

launches_per_year = useful_data.groupby('Year')['Organisation'].count().reset_index()

launches_per_year.columns = ['Year', 'Number of Launches'] #rename

fig = px.bar(
    launches_per_year,
    x='Year',
    y='Number of Launches',
    title='Número de Lançamentos por Ano',
    labels={'Number of Launches': 'Número de Lançamentos', 'Year': 'Ano'},
    text='Number of Launches'
)

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_tickformat=",.0f",
)
fig.show()

## Chart of the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time?

In [ ]:
useful_data['Date'] = pd.to_datetime(useful_data['Date'], errors='coerce')
useful_data = useful_data.dropna(subset=['Date'])

useful_data['Year_Month'] = useful_data['Date'].dt.to_period('M').astype(str)
launches_per_month = useful_data.groupby('Year_Month')['Organisation'].count().reset_index()

launches_per_month.columns = ['Year_Month', 'Number of Launches']
launches_per_month['Rolling Average'] = launches_per_month['Number of Launches'].rolling(window=12).mean() #smooth mean

fig = px.line(
    launches_per_month,
    x='Year_Month',
    y=['Number of Launches', 'Rolling Average'],
    title='Número de Lançamentos Mês a Mês até o Presente com Média Móvel',
    labels={'Number of Launches': 'Número de Lançamentos', 'Year_Month': 'Ano-Mês'},
    markers=True
)

max_launch_month = launches_per_month.loc[launches_per_month['Number of Launches'].idxmax()] #destacar
fig.add_annotation(
    x=max_launch_month['Year_Month'],
    y=max_launch_month['Number of Launches'],
    text=f"Maior Lançamento: {max_launch_month['Year_Month']} ({max_launch_month['Number of Launches']} lançamentos)",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40
)
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_tickformat=",.0f",  #whole number
)
fig.show()

##Which months are most popular and least popular for launches?
Which time of year seems to be best for space missions?

In [ ]:
launches_per_month = useful_data.groupby(useful_data['Date'].dt.to_period('M')).size().reset_index(name='Number of Launches')
launches_per_month['Date'] = launches_per_month['Date'].dt.to_timestamp()
launches_per_month['Month'] = launches_per_month['Date'].dt.month_name()

launches_per_month_by_month = launches_per_month.groupby('Month')['Number of Launches'].sum().reset_index()

launches_per_month_by_month['Month'] = pd.Categorical(launches_per_month_by_month['Month'],
                                                      categories=["January", "February", "March", "April", "May", "June",
                                                                  "July", "August", "September", "October", "November", "December"],
                                                      ordered=True)
launches_per_month_by_month = launches_per_month_by_month.sort_values('Month')

fig = px.bar(
    launches_per_month_by_month,
    x='Month',
    y='Number of Launches',
    title='Número de Lançamentos por Mês',
    labels={'Number of Launches': 'Número de Lançamentos', 'Month': 'Mês'},
    text_auto='.2s'
)
launches_per_month_by_month['Rolling Average'] = launches_per_month_by_month['Number of Launches'].rolling(window=3).mean()

fig.add_scatter(
    x=launches_per_month_by_month['Month'],
    y=launches_per_month_by_month['Rolling Average'],
    mode='lines+markers',
    name='Média Móvel 3 meses',
    line=dict(color='orange', dash='dot')
)

fig.show()

most_popular_month = launches_per_month_by_month.loc[launches_per_month_by_month['Number of Launches'].idxmax()]
least_popular_month = launches_per_month_by_month.loc[launches_per_month_by_month['Number of Launches'].idxmin()]
print(f'Mês mais popular: {most_popular_month["Month"]} com {most_popular_month["Number of Launches"]} lançamentos')
print(f'Mês menos popular: {least_popular_month["Month"]} com {least_popular_month["Number of Launches"]} lançamentos')

## How has the Launch Price varied Over Time?

Line chart that shows the average anual price of rocket launches over time.

In [ ]:
contain_price_df['Date'] = pd.to_datetime(contain_price_df['Date'], errors='coerce')
contain_price_df['Price'] = pd.to_numeric(contain_price_df['Price'], errors='coerce')

contain_price_df.loc[:, 'Price'] = contain_price_df['Price'].fillna(contain_price_df['Price'].mean())#Preenche valores nulos na coluna 'Price' com a media

contain_price_df['Year'] = contain_price_df['Date'].dt.year

average_price_per_year = contain_price_df.groupby('Year')['Price'].mean().reset_index()

fig = px.line(
    average_price_per_year,
    x='Year',
    y='Price',
    title='Variação do Preço Médio dos Lançamentos por Ano',
    labels={'Price': 'Preço Médio Anual (em milhões de dólares)', 'Year': 'Ano'},
)
fig.show()

## Chart of the Number of Launches over Time by the Top 10 Organisations.

How has the dominance of launches changed over time between the different players?

In [ ]:
useful_data['Date'] = pd.to_datetime(useful_data['Date'], errors='coerce')
useful_data['Year'] = useful_data['Date'].dt.year

launches_per_year = useful_data.groupby(['Organisation', 'Year']).size().reset_index(name='Number of Launches')

# Obter as top 10 orgs com mais lançamentos
top_10_organisations = launches_per_year.groupby('Organisation')['Number of Launches'].sum().nlargest(10).index
filtered_data = launches_per_year[launches_per_year['Organisation'].isin(top_10_organisations)]

fig = px.line(
    filtered_data,
    x='Year',
    y='Number of Launches',
    color='Organisation',
    title='Número de Lançamentos ao Longo do Tempo pelas Top 10 Organizações',
    labels={'Number of Launches': 'Número de Lançamentos', 'Year': 'Ano'},
    markers=True
)
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=True
)
fig.show()

## Cold War Section

The cold war lasted from the start of the dataset up until 1991.

###Pie Chart comparing the total number of launches of the USSR and the USA


In [ ]:
useful_data['Date'] = pd.to_datetime(useful_data['Date'], errors='coerce')
# Filtrar os dados para o período da Guerra Fria (1957 a 1991)
cold_war_data = useful_data[(useful_data['Date'].dt.year >= 1957) & (useful_data['Date'].dt.year <= 1991)]

# Substituir os nomes
cold_war_data['Organisation'] = cold_war_data['Organisation'].replace({
    'Soviet Union': 'USSR',
    'Kazakhstan': 'USSR',
    'RVSN USSR': 'USSR',
    'Armed Forces of the USSR': 'USSR',
    'USSR': 'USSR',
    'OKB-586':'USSR',
    'General Dynamics': 'USA',
    'US Air Force': 'USA',
    'NASA' : 'USA',
    'Martin Marietta': 'USA',
    'Arianespace' : 'EU',
    'CASC' : 'CHN',
    'MHI' : 'JPN',
    'Boeing': 'USA',
    'ISAS': 'JPN',
    'US Navy': 'USA',
    'ESA' : 'EU',
    'ASI' : 'EU',
    'CNES': 'EU',
    'AMBA': 'USA',
    'ISRO': 'IND',
    'CECLES': 'EU',
    "Arm??e de l'Air": 'EU',
    'Northrop':'USA',
    'Yuzhmash':'USSR',
    'RAE':'EU',
    'Douglas': 'USA',
    'Lockheed': 'USA',
    'Roscosmos': 'USSR',
    'UT' : 'USA',
})

cold_war_data['Year'] = cold_war_data['Date'].dt.year

#apenas URSS e USA
filtered_cold_war = cold_war_data[cold_war_data['Organisation'].isin(['USSR', 'USA'])]

total_launches_cold_war = filtered_cold_war['Organisation'].value_counts().reset_index()
total_launches_cold_war.columns = ['Organisation', 'Total Launches']

fig_pie = px.pie(
    total_launches_cold_war,
    names='Organisation',
    values='Total Launches',
    title='Participação Total de Lançamentos: URSS vs EUA (1957–1991)',
    labels={'Total Launches': 'Número Total de Lançamentos'}
)
fig_pie.show()

### Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [ ]:
useful_data['Date'] = pd.to_datetime(useful_data['Date'], errors='coerce')

useful_data['Organisation'] = useful_data['Organisation'].replace({
    'Soviet Union': 'USSR',
    'Kazakhstan': 'USSR',
    'RVSN USSR': 'USSR',
    'Armed Forces of the USSR': 'USSR',
    'OKB-586': 'USSR',
    'Yuzhmash': 'USSR',
    'Roscosmos': 'USSR',

    'General Dynamics': 'USA',
    'US Air Force': 'USA',
    'NASA': 'USA',
    'Martin Marietta': 'USA',
    'Boeing': 'USA',
    'US Navy': 'USA',
    'AMBA': 'USA',
    'Northrop': 'USA',
    'Douglas': 'USA',
    'Lockheed': 'USA',
    'UT': 'USA'
})

superpowers = useful_data[useful_data['Organisation'].isin(['USSR', 'USA'])]

#period of time
cold_war_superpowers = superpowers[
    (superpowers['Date'].dt.year >= 1957) &
    (superpowers['Date'].dt.year <= 1991)
]

cold_war_superpowers['Year'] = cold_war_superpowers['Date'].dt.year
launches_per_year = cold_war_superpowers.groupby(['Year', 'Organisation']).size().reset_index(name='Launch Count')

fig = px.line(
    launches_per_year,
    x='Year',
    y='Launch Count',
    color='Organisation',
    markers=True,
    title='Lançamentos Anuais: URSS vs EUA (1957–1991)',
    labels={'Launch Count': 'Número de Lançamentos', 'Year': 'Ano'}
)
fig.show()

### Chart of the Total Number of Mission Failures Year on Year.

In [ ]:
useful_data['Date'] = pd.to_datetime(useful_data['Date'], errors='coerce')
useful_data['Organisation'] = useful_data['Organisation'].replace({
    'Soviet Union': 'USSR',
    'Kazakhstan': 'USSR',
    'RVSN USSR': 'USSR',
    'Armed Forces of the USSR': 'USSR',
    'OKB-586': 'USSR',
    'Yuzhmash': 'USSR',
    'Roscosmos': 'USSR',

    'General Dynamics': 'USA',
    'US Air Force': 'USA',
    'NASA': 'USA',
    'Martin Marietta': 'USA',
    'Boeing': 'USA',
    'US Navy': 'USA',
    'AMBA': 'USA',
    'Northrop': 'USA',
    'Douglas': 'USA',
    'Lockheed': 'USA',
    'UT': 'USA'
})
superpowers = useful_data[useful_data['Organisation'].isin(['USSR', 'USA'])]

cold_war_superpowers = superpowers[
    (superpowers['Date'].dt.year >= 1957) &
    (superpowers['Date'].dt.year <= 1991)
]

cold_war_superpowers['Year'] = cold_war_superpowers['Date'].dt.year

failures = cold_war_superpowers[cold_war_superpowers['Mission_Status'] != 'Success']

failures_per_year = failures.groupby(['Year', 'Organisation']).size().reset_index(name='Failures')

fig = px.line(
    failures_per_year,
    x='Year',
    y='Failures',
    color='Organisation',
    markers=True,
    title='Falhas em Missões: URSS vs EUA (1957–1991)',
    labels={'Failures': 'Falhas', 'Year': 'Ano'}
)
fig.show()

### Chart of the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time?

In [ ]:
total_launches = cold_war_superpowers.groupby(['Year', 'Organisation']).size().reset_index(name='Total')

failures = cold_war_superpowers[cold_war_superpowers['Mission_Status'] != 'Success']
failures_per_year = failures.groupby(['Year', 'Organisation']).size().reset_index(name='Failures')

merged = pd.merge(total_launches, failures_per_year, on=['Year', 'Organisation'], how='left')
merged['Failures'] = merged['Failures'].fillna(0)
merged['Failure_Rate (%)'] = (merged['Failures'] / merged['Total']) * 100

fig = px.line(
    merged,
    x='Year',
    y='Failure_Rate (%)',
    color='Organisation',
    markers=True,
    title='Percentual de Falhas por Ano: URSS vs EUA (1957–1991)',
    labels={'Failure_Rate (%)': 'Taxa de Falha (%)', 'Year': 'Ano'}
)
fig.show()

### For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including 2020

Do the results change if we only look at the number of successful launches?

In [ ]:
successes = superpowers[superpowers['Mission_Status'] == 'Success']

success_launches = successes.groupby(['Year', 'Organisation']).size().reset_index(name='Successful Launches')

leaders_success = success_launches.sort_values(['Year', 'Successful Launches'], ascending=[True, False]).drop_duplicates('Year')

success_launches = successes.groupby(['Year', 'Organisation']).size().reset_index(name='Successful Launches')

fig_success = px.line(
    success_launches,
    x='Year',
    y='Successful Launches',
    color='Organisation',
    markers=True,
    title='Lançamentos Bem-Sucedidos por Ano — URSS vs EUA',
    labels={'Successful Launches': 'Lançamentos com Sucesso', 'Year': 'Ano'}
)
fig_success.show()

###Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020?

In [ ]:
launches_per_year = superpowers.groupby(['Year', 'Organisation']).size().reset_index(name='Launches')

fig = px.line(
    launches_per_year,
    x='Year',
    y='Launches',
    color='Organisation',
    markers=True,
    title='Lançamentos por Ano por Organização',
    labels={'Launches': 'Número de Lançamentos', 'Year': 'Ano'}
)
fig.show()